In [28]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import  confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_excel(r"C:\Users\RameshMisale\Documents\Profile_data.xlsx")
df1 = df
df1['Resubmit'] = pd.to_datetime(df1['Resubmit'])
df1['Resubmit'] = df1['Resubmit'].notnull().astype(int)
columns_to_fill = ['rcra_non_haz_exempt','halogens_flag','no_reactivity_flag','layered','viscosity','odor_flag','ph_flag',
    'flash_point_flag','boiling_point_flag','btu_per_lbs','pumpable_waste_flag','polymerizable_flag','benzene_waste_flag',
    'voc_100_ppm','marine_pollutant_flag','origin_code','sds_attached','specific_gravity','benzene_section_flag',
    'max_benzene_flag','benzene_water','prohibited_land_disposal','uts_waste','voc_500_ppm','specialpricing_flag',
    'intercompany_flag','mgp_flag','pa_waste_catogory','debris','compressed_gas','analytical_ind',
    'generatorknowledge_ind','sds_ind','formulary_attached','analytical_attached','sample_provided','mgplock_flag',
    'naics_flag','federal_universal_waste','generator_state_universal_waste']
df1[columns_to_fill] = df1[columns_to_fill].fillna(0, inplace=False)

col = ['water_percentage', 'toc_percentage'] #filling with mean for these 2 columns
df1[col] = df1[col].fillna(df1[col].mean(), inplace=False)
#df1.isnull().sum()
object_columns = df1.select_dtypes(include=['object','datetime64']).columns
df_dropped_objects = df1.drop(object_columns, axis=1)
# df_dropped_objects.dtypes

#It will remove the zero variance features

var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df_dropped_objects)

constant_columns = [column for column in df_dropped_objects.columns
                    if column not in df_dropped_objects.columns[var_thres.get_support()]]
print(f"Number of constant columns: {len(constant_columns)}")
for feature in constant_columns:
    unique_values = df_dropped_objects[feature].unique()
    print(f"Constant column: {feature}")
    print(f"Unique values: {unique_values}")
    print("\n")
data = df_dropped_objects.drop(constant_columns,axis=1) #dropped the columns here 

def correlation(dataset,threshold):
    col_corr=set()
    corr_matrix=dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j]>threshold):
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr


corr_features=correlation(data,0.8)
len(set(corr_features))


data1=data.drop(corr_features,axis=1) # dropped the correlated features here


threshold_percentage = 90 
null_percentage = (data1.isnull().sum() / len(data1)) * 100
columns_to_drop = null_percentage[null_percentage > threshold_percentage].index
data1_dropped = data1.drop(columns=columns_to_drop)
#data1_dropped.isnull().sum()

data1_dropped = data1_dropped.fillna(0) #filling the null values with 0
#data1_dropped.isnull().sum() 

dff = data1_dropped.drop(columns=['ReturnCount','DaysAssignReadyForGenSign','profile_id','web_profile_number',
                                  'DaysSubmitToAssign','DaysAssignReadyForGenSign','DaysReadyForGenSignSentForGenSign',
                                  'DaysDocSignReturnedToApproved','container_type_id','DaysInitiatedToSubmitted',
'DaysSentForGenSignToDocSignReturned','vendor_id','ContractID','ldr_class_id','CustomerId','CollectionId',
'HCSId','Recert','is_template_profile_flag','AssignUser_id','status_code_id','source_code_id','form_code_id',
'management_method_code_id','outbound_profile_id','price_type_code_id','parent_profile_id','health_chemical_identity_id',
'flammability_chemical_identity_id','reactivity_chemical_identity_id','process_code_id',
'SalesrepID','InternalCoordinatorID','MarketDriverID','InsideSalesRepID','requested_process_code_id'],axis=1)
dff.shape

vif_data = pd.DataFrame()
vif_data["feature"] = dff.columns
vif_data["VIF"] = [variance_inflation_factor(dff.values, i) for i in range(dff.shape[1])]
print(vif_data)

high_vif_columns = vif_data[vif_data["VIF"] > 5]["feature"].tolist()
high_vif_columns

df_final = dff.drop(columns=high_vif_columns)
df_final.shape

XX = df_final.drop('Resubmit', axis='columns')
yy= df_final['Resubmit']
XX_train, XX_test, yy_train, yy_test = train_test_split(XX,yy, test_size=0.2,random_state=42,stratify=yy)

count_0_class,count_1_class = df_final.Resubmit.value_counts()
df_class_0 = df_final[df_final["Resubmit"]==0]
df_class_1 = df_final[df_final["Resubmit"]==1]
X_train, X_test, y_train, y_test = train_test_split(XX,yy, test_size=0.2,random_state=15,stratify=yy)

df_class_0_under = df_class_0.sample(count_1_class)
df_test_under = pd.concat([df_class_0_under, df_class_1],axis=0)

print('Random Under-sampling:')
print(df_test_under.Resubmit.value_counts())

XX = df_test_under.drop('Resubmit', axis='columns')
yy= df_test_under['Resubmit']
XX_train, XX_test, yy_train, yy_test = train_test_split(XX,yy, test_size=0.2,random_state=42,stratify=yy)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
logistic_regression = LogisticRegression()
logistic_regression.fit(XX_train, yy_train)
y_pred_lr_train = logistic_regression.predict(XX_train)
y_pred_lr_test = logistic_regression.predict(XX_test)

# Accuracy
accuracy_lr_train = accuracy_score(yy_train, y_pred_lr_train)
accuracy_lr_test = accuracy_score(yy_test, y_pred_lr_test)
print(f'Logistic Regression Training Accuracy: {accuracy_lr_train}')
print(f'Logistic Regression Testing Accuracy: {accuracy_lr_test}')

# # classification report
# print(classification_report(yy_test, y_pred_lr_test))
# print(confusion_matrix(yy_test, y_pred_lr_test))

precision_lr_test = precision_score(yy_test, y_pred_lr_test)
recall_lr_test = recall_score(yy_test, y_pred_lr_test)
f1_lr_test = f1_score(yy_test, y_pred_lr_test)

# Display precision, recall, and f1-score
print(f'\nPrecision: {precision_lr_test:.4f}')
print(f'Recall: {recall_lr_test:.4f}')
print(f'F1-Score: {f1_lr_test:.4f}')


# import pickle
# file_path = (r'C:\Users\RameshMisale\Downloads\Harsco_model.pkl')

# # Open the file in binary write mode
# with open(file_path, 'wb') as file:
#     # Use pickle.dump() to serialize and save the data to the file
#     pickle.dump(file_path, file)

# print(f'Data saved to {file_path}')

In [35]:
with open(file_path, 'wb') as file:
    pickle.dump(logistic_regression, file)

In [40]:
import joblib
model_case = joblib.load(open(r'C:\Users\RameshMisale\Downloads\Harsco_model.pkl', 'rb'))

In [41]:
import joblib

# Assuming 'model_case' is your trained model
# Save the model to a file using joblib
joblib.dump(model_case, r'C:\Users\RameshMisale\Downloads\Harsco_model.pkl')


['C:\\Users\\RameshMisale\\Downloads\\Harsco_model.pkl']

In [42]:
df_final.to_csv('your_file_path.csv', index=False)


In [43]:
df_final.to_csv(r'C:\Users\RameshMisale\Downloads\df_final.csv', index=False)
